In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("..")

In [3]:
from constrerl.evaluate import (
    eval_submission_6_1_NER,
    eval_submission_6_3_ternary_tag_RE,
    eval_submission_6_4_ternary_mention_RE,
    eval_submission_6_2_binary_tag_RE,
)
from constrerl.erl_schema import convert_to_output, Article
import glob
from pathlib import Path
import json
import pandas as pd
from collections.abc import Callable, Awaitable


In [4]:
results_dir = "../data/merge_dev"
ground_truth_file = "../data/annotations/dev/dev.json"
report_dir = Path("report_ontug")
results_dir = Path(results_dir)
ner_result_dir = Path("../data/results_ner_dev")
ground_truth_file = Path(ground_truth_file)
lbl_xtra = ":ontug"


top_results = 0

k_limits = [None]

with open(ground_truth_file) as f:
    ground_truth = json.load(f)

In [20]:
eval_results: list[dict] = []

import re


def scoring_to_dict(
    predictions: dict, eval_f: Callable[[dict, dict], tuple[float]], gt=ground_truth
) -> dict:
    precision, recall, f1, micro_precision, micro_recall, micro_f1 = eval_f(
        predictions, gt
    )
    return {
        "$P$": precision,
        "$R$": recall,
        "$F_1$": f1,
        "$P_{micro}$": micro_precision,
        "$R_{micro}$": micro_recall,
        "$F_{1,micro}$": micro_f1,
    }


def scoring_to_df(
    eval_f: Callable[[str | Path, dict], tuple[float]],
    key: str,
    res_dir=results_dir,
    k_limits_set=None,
    cap_gt=False,
) -> pd.DataFrame:
    eval_results: list[dict] = []
    merge_mode = "merge" in str(res_dir)
    further_mode = not (("old" in str(res_dir)) or ("merge" in str(res_dir)))
    print(f"Further mode: {further_mode}")
    files = res_dir.glob("*.json")
    if not res_dir.is_dir():
        files = [res_dir]
    for result_file in files:
        set_mode = False
        result_file = Path(result_file)
        try:
            with open(result_file, "r", encoding="utf-8") as file:
                predictions: dict = json.load(file)
        except OSError:
            raise OSError(f"Error in opening the specified json file: {result_file}")

        if k_limits_set is None or len(k_limits_set) == 0:
            k_list = [None]
        else:
            k_list = k_limits_set
        for k in k_list:
            predictions_limited = predictions.copy()
            predictions_limited = {
                id: {key: predictions_limited[id][key][:k]}
                for id in predictions_limited.keys()
            }
            k_gt: dict = ground_truth.copy()
            if cap_gt:
                k_gt = {id: {key: k_gt[id][key][:k]} for id in k_gt.keys()}
            if "union" in result_file.name or "intersection" in result_file.name:
                set_mode = True
            set_op = "$\cup$" if "union" in result_file.name else "$\cap$"
            eval_result = scoring_to_dict(predictions_limited, eval_f, gt=k_gt)
            model_name = result_file.name.rstrip(".json")
            if set_mode:
                model_name = model_name.rstrip("_intersection").rstrip("_union")
            model_name = (
                " ".join(model_name.rstrip(".json").split("-")[:2])
                if "openai" not in model_name
                else " ".join(model_name.rstrip(".json").split("-")[:3])
            )
            # capitalize the first letter of the name
            model_name = " ".join(
                [
                    word.capitalize() if i == 0 else word
                    for i, word in enumerate(model_name.split(" "))
                ]
            )
            model_name = re.sub(r"(\d)b", "\\1B", model_name)
            splits = model_name.split(" ")
            if len(splits) > 2:
                model_name = splits[0] + " " + "-".join(splits[1:])
            low_tokens = "low-tokens" in result_file.name
            high_tokens = "high-tokens" in result_file.name
            entity_labels = "entity-labels" in result_file.name
            result_dict = {
                "Model": model_name,
                "RAG": "\checkmark" if "rag" in result_file.name else "$\\times$",
                "Reorder": "\checkmark"
                if "reorder" in result_file.name
                else "$\\times$",
                # "Low Tokens": "\checkmark"
                # if "low-tokens" in result_file.name
                # else "$\\times$",
                # "Entity Labels": "\checkmark"
                # if "entity-labels" in result_file.name
                # else "$\\times$",
            }
            if further_mode:
                result_dict["LoRA+"] = (
                    "\checkmark" if "long" in result_file.name else "$\\times$"
                )
                result_dict["Low $t$"] = "\checkmark" if low_tokens else "$\\times$"
                result_dict["High $t$"] = "\checkmark" if high_tokens else "$\\times$"
                result_dict["Entities"] = "\checkmark" if entity_labels else "$\\times$"
                if k_limits_set is not None and len(k_limits_set) > 0:
                    result_dict["$k$"] = k if k is not None else "$\\times$"
            else:
                result_dict["LoRA"] = (
                    "\checkmark" if "lora" in result_file.name else "$\\times$"
                )
            if set_mode:
                result_dict["Set"] = set_op
            result_dict.update(eval_result)
            # result_dict.update({f"6_2_2_{k}": v for k, v in ternary_tag_score.items()})
            # result_dict.update({f"6_2_3_{k}": v for k, v in ternary_mention_score.items()})
            eval_results.append(result_dict)
    eval_df = pd.DataFrame(eval_results)
    if further_mode and "$k$" not in eval_df.columns:
        eval_df = eval_df[
            (eval_df["Low $t$"] == "\checkmark")
            | (eval_df["Entities"] == "\checkmark")
            | (eval_df["LoRA+"] == "\checkmark")
        ]
    if merge_mode:
        eval_df = eval_df[eval_df["Model"].str.contains("Hermes 3B")]
    valid_cols = [
        c
        for c in [
            "Set",
            "Model",
            "RAG",
            "LoRA",
            "LoRA+",
            "Reorder",
            "Low $t$",
            "High $t$",
            "Entities",
            "$k$",
        ]
        if c in eval_df.columns
    ]
    eval_df.set_index(valid_cols, inplace=True)
    eval_df = eval_df.sort_index()
    # if "$F_{1,micro}$" in eval_df.columns:
    #     eval_df = eval_df.sort_values("$F_{1,micro}$")
    return eval_df


task_6_1_1_df = scoring_to_df(
    eval_submission_6_1_NER, "entities", res_dir=ner_result_dir
)
task_6_2_1_df = scoring_to_df(
    eval_submission_6_2_binary_tag_RE, "binary_tag_based_relations"
)
task_6_2_2_df = scoring_to_df(
    eval_submission_6_3_ternary_tag_RE, "ternary_tag_based_relations"
)
task_6_2_3_df = scoring_to_df(
    eval_submission_6_4_ternary_mention_RE, "ternary_mention_based_relations"
)

Further mode: True
=== Removed 458 duplicated entities from predictions ===
=== Removed 111 overlapping entities ===
=== Removed 335 duplicated entities from predictions ===
=== Removed 86 overlapping entities ===
=== Removed 89 duplicated entities from predictions ===
=== Removed 35 overlapping entities ===
=== Removed 458 duplicated entities from predictions ===
=== Removed 111 overlapping entities ===
=== Removed 399 duplicated entities from predictions ===
=== Removed 61 overlapping entities ===
=== Removed 231 duplicated entities from predictions ===
=== Removed 82 overlapping entities ===
=== Removed 101 duplicated entities from predictions ===
=== Removed 48 overlapping entities ===
=== Removed 85 duplicated entities from predictions ===
=== Removed 64 overlapping entities ===
=== Removed 231 duplicated entities from predictions ===
=== Removed 82 overlapping entities ===
=== Removed 92 duplicated entities from predictions ===
=== Removed 61 overlapping entities ===
=== Removed 

<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:86: SyntaxWarning: invalid escape sequence '\c'
<>:87: SyntaxWarning: invalid escape sequence '\c'
<>:99: SyntaxWarning: invalid escape sequence '\c'
<>:101: SyntaxWarning: invalid escape sequence '\c'
<>:102: SyntaxWarning: invalid escape sequence '\c'
<>:103: SyntaxWarning: invalid escape sequence '\c'
<>:108: SyntaxWarning: invalid escape sequence '\c'
<>:119: SyntaxWarning: invalid escape sequence '\c'
<>:120: SyntaxWarning: invalid escape sequence '\c'
<>:121: SyntaxWarning: invalid escape sequence '\c'
<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:60: SyntaxWarning: invalid escape sequence '\c'
<>:86: SyntaxWarning: invalid escape sequence '\c'
<>:87: SyntaxWarning: invalid escape sequence '\c'
<>:99: SyntaxWarning: invalid escape sequence '\c'
<>:101: SyntaxWarning: invalid escape sequence '\c'
<>:102: SyntaxWarning: invalid escape sequence '\c'
<>:103: SyntaxWarning:

In [15]:
task_6_1_1_df[top_results:].style.highlight_max(axis=0, props="textbf:--rwrap;").format(precision=2).to_latex(
    report_dir / "task_6_1_1.tex",
    # float_format="%.2f",
    caption="Dev Set Result for Task 6.1.1 (NER) for various models and approaches.",
    label=f"tab:task:6_1_1{lbl_xtra}",
    clines="all;data",
    hrules=True
)
task_6_1_1_df

$P$  \
Model     RAG        LoRA+    Reorder    Low $t$    High $t$ Entities               
Hermes 3B $\times$   $\times$ $\times$   $\times$   $\times$ \checkmark  0.048768   
                                         \checkmark $\times$ $\times$    0.015713   
                                                             \checkmark  0.045376   
                              \checkmark $\times$   $\times$ \checkmark  0.048768   
                                         \checkmark $\times$ \checkmark  0.045376   
          \checkmark $\times$ $\times$   $\times$   $\times$ \checkmark  0.212756   
                                         \checkmark $\times$ $\times$    0.318396   
                                                             \checkmark  0.252185   
                              \checkmark $\times$   $\times$ \checkmark  0.212756   
                                         \checkmark $\times$ \checkmark  0.252185   

                                                                              $R$  \
Model     RAG        LoRA+    Reorder    Low $t$    High $t$ Entities               
Hermes 3B $\times$   $\times$ $\times$   $\times$   $\times$ \checkmark  0.018770   
                                         \checkmark $\times$ $\times$    0.017106   
                                                             \checkmark  0.016660   
                              \checkmark $\times$   $\times$ \checkmark  0.018770   
                                         \checkmark $\times$ \checkmark  0.016660   
          \checkmark $\times$ $\times$   $\times$   $\times$ \checkmark  0.081851   
                                         \checkmark $\times$ $\times$    0.066426   
                                                             \checkmark  0.054985   
                              \checkmark $\times$   $\times$ \checkmark  0.081851   
                                         \checkmark $\times$ \checkmark  0.054985   

                                                                            $F_1$  \
Model     RAG        LoRA+    Reorder    Low $t$    High $t$ Entities               
Hermes 3B $\times$   $\times$ $\times$   $\times$   $\times$ \checkmark  0.025532   
                                         \checkmark $\times$ $\times$    0.013724   
                                                             \checkmark  0.022781   
                              \checkmark $\times$   $\times$ \checkmark  0.025532   
                                         \checkmark $\times$ \checkmark  0.022781   
          \checkmark $\times$ $\times$   $\times$   $\times$ \checkmark  0.112518   
                                         \checkmark $\times$ $\times$    0.099484   
                                                             \checkmark  0.083981   
                              \checkmark $\times$   $\times$ \checkmark  0.112518   
                                         \checkmark $\times$ \checkmark  0.083981   

                                                                         $P_{micro}$  \
Model     RAG        LoRA+    Reorder    Low $t$    High $t$ Entities                  
Hermes 3B $\times$   $\times$ $\times$   $\times$   $\times$ \checkmark     0.055046   
                                         \checkmark $\times$ $\times$       0.037415   
                                                             \checkmark     0.057221   
                              \checkmark $\times$   $\times$ \checkmark     0.055046   
                                         \checkmark $\times$ \checkmark     0.057221   
          \checkmark $\times$ $\times$   $\times$   $\times$ \checkmark     0.250000   
                                         \checkmark $\times$ $\times$       0.292500   
                                                             \checkmark     0.308970   
                              \checkmark $\times$   $\times$ \checkmark     0.250000   
                                         \check

In [16]:
task_6_2_1_df

$P$       $R$  \
Set    Model          RAG        LoRA       Reorder                          
$\cap$ Hermes 3B      $\times$   $\times$   $\times$    0.033333  0.011111   
                                            \checkmark  0.011111  0.003704   
                                 \checkmark $\times$    0.177778  0.040509   
                                            \checkmark  0.177778  0.040509   
                      \checkmark $\times$   $\times$    0.251634  0.071040   
                                            \checkmark  0.133951  0.050731   
                                 \checkmark $\times$    0.173016  0.040553   
                                            \checkmark  0.145556  0.040201   
       Hermes 8B      $\times$   $\times$   $\times$    0.055556  0.007326   
                                            \checkmark  0.055556  0.007326   
                      \checkmark $\times$   $\times$    0.288360  0.100924   
                                            \checkmark  0.128704  0.053534   
       Openai 4-1     $\times$   $\times$   $\times$    0.122650  0.033526   
                                            \checkmark  0.105185  0.037642   
                      \checkmark $\times$   $\times$    0.274799  0.106664   
                                            \checkmark  0.191799  0.086574   
       Openai 4o-m    $\times$   $\times$   $\times$    0.096085  0.033244   
       Openai 4o-mini $\times$   $\times$   \checkmark  0.201587  0.070992   
                      \checkmark $\times$   $\times$    0.231246  0.104352   
                                            \checkmark  0.138179  0.066680   
$\cup$ Hermes 3B      $\times$   $\times$   $\times$    0.546896  0.530845   
                                            \checkmark  0.537235  0.530845   
                                 \checkmark $\times$    0.594853  0.539063   
                                            \checkmark  0.594853  0.539063   
                      \checkmark $\times$   $\times$    0.527807  0.532933   
                                            \checkmark  0.556180  0.531523   
                                 \checkmark $\times$    0.585411  0.527686   
                                            \checkmark  0.589516  0.537624   
       Hermes 8B      $\times$   $\times$   $\times$    0.547353  0.527914   
                                            \checkmark  0.547353  0.527914   
                      \checkmark $\times$   $\times$    0.541338  0.545648   
                                            \checkmark  0.545934  0.549694   
       Openai 4-1     $\times$   $\times$   $\times$    0.468689  0.593778   
                                            \checkmark  0.436586  0.539969   
                      \checkmark $\times$   $\times$    0.486577  0.577633   
                                            \checkmark  0.466222  0.592491   
       Openai 4o-m    $\times$   $\times$   $\times$    0.393308  0.576556   
       Openai 4o-mini $\times$   $\times$   \checkmark  0.400598  0.567268   
                      \checkmark $\times$   $\times$    0.489746  0.596114   
                                            \checkmark  0.476112  0.560064   

                                                           $F_1$  $P_{micro}$  \
Set    Model          RAG        LoRA       Reorder                             
$\cap$ Hermes 3B      $\times$   $\times$   $\times$    0.016667     0.666667   
                                            \checkmark  0.005556     0.500000   
                                 \checkmark $\times$    0.062239     0.961538   
                                            \checkmark  0.062239     0.961538   
                      \checkmark $\times$   $\times$    0.104302     0.846154   
                                            \checkmark  0.067101     0.692308   
                                 \checkmark $\times$    0.061778     0.888889   
                                            \checkmark 

In [21]:
task_6_2_1_df[top_results:].style.highlight_max(axis=0, props="textbf:--rwrap;").format(precision=2).to_latex(
    report_dir / "task_6_2_1.tex",
    # float_format="%.2f",
    caption="Dev Set Result for Task 6.2.1 for various models and approaches.",
    label=f"tab:task:6_2_1{lbl_xtra}",
    clines="all;data",
    hrules=True
)
task_6_2_1_df

$P$       $R$  \
Set    Model     RAG        LoRA       Reorder                          
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.033333  0.011111   
                                       \checkmark  0.011111  0.003704   
                            \checkmark $\times$    0.177778  0.040509   
                                       \checkmark  0.177778  0.040509   
                 \checkmark $\times$   $\times$    0.251634  0.071040   
                                       \checkmark  0.133951  0.050731   
                            \checkmark $\times$    0.173016  0.040553   
                                       \checkmark  0.145556  0.040201   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.546896  0.530845   
                                       \checkmark  0.537235  0.530845   
                            \checkmark $\times$    0.594853  0.539063   
                                       \checkmark  0.594853  0.539063   
                 \checkmark $\times$   $\times$    0.527807  0.532933   
                                       \checkmark  0.556180  0.531523   
                            \checkmark $\times$    0.585411  0.527686   
                                       \checkmark  0.589516  0.537624   

                                                      $F_1$  $P_{micro}$  \
Set    Model     RAG        LoRA       Reorder                             
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.016667     0.666667   
                                       \checkmark  0.005556     0.500000   
                            \checkmark $\times$    0.062239     0.961538   
                                       \checkmark  0.062239     0.961538   
                 \checkmark $\times$   $\times$    0.104302     0.846154   
                                       \checkmark  0.067101     0.692308   
                            \checkmark $\times$    0.061778     0.888889   
                                       \checkmark  0.058130     0.857143   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.510897     0.652381   
                                       \checkmark  0.508799     0.649289   
                            \checkmark $\times$    0.543255     0.706468   
                                       \checkmark  0.543255     0.706468   
                 \checkmark $\times$   $\times$    0.506510     0.630631   
                                       \checkmark  0.516314     0.620536   
                            \checkmark $\times$    0.533693     0.683168   
                                       \checkmark  0.540276     0.692683   

                                                   $R_{micro}$  $F_{1,micro}$  
Set    Model     RAG        LoRA       Reorder                                 
$\cap$ Hermes 3B $\times$   $\times$   $\times$       0.009091       0.017937  
                                       \checkmark     0.004545       0.009009  
                            \checkmark $\times$       0.113636       0.203252  
                                       \checkmark     0.113636       0.203252  
                 \checkmark $\times$   $\times$       0.150000       0.254826  
                                       \checkmark     0.122727       0.208494  
                            \checkmark $\times$       0.109091       0.194332  
                                       \checkmark     0.109091       0.193548  
$\cup$ Hermes 3B $\times$   $\times$   $\times$       0.622727       0.637209  
                                       \checkmark     0.622727       0.635731  
                            \checkmark $\times$       0.645455       0.674584  
                                       \checkmark     0.645455       0.674584  
                 \checkmark $\times$   $\times$       0.636364       0.633484  
                                       \checkmark     0.631818       0.626126  
                            \checkmark $\times$       0.627273       0.654028  
            

In [22]:
task_6_2_2_df[top_results:].style.highlight_max(axis=0, props="textbf:--rwrap;").format(precision=2).to_latex(
    report_dir / "task_6_2_2.tex",
    # float_format="%.2f",
    caption="Further Dev Set Result for Task 6.2.2 for various models and approaches.",
    label=f"tab:task:6_2_2{lbl_xtra}",
    clines="all;data",
    hrules=True
)
task_6_2_2_df

$P$       $R$  \
Set    Model     RAG        LoRA       Reorder                          
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.031250  0.010417   
                                       \checkmark  0.010417  0.003472   
                            \checkmark $\times$    0.166667  0.038863   
                                       \checkmark  0.166667  0.038863   
                 \checkmark $\times$   $\times$    0.235907  0.066895   
                                       \checkmark  0.125579  0.047856   
                            \checkmark $\times$    0.162202  0.039199   
                                       \checkmark  0.136458  0.038574   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.547810  0.526916   
                                       \checkmark  0.537133  0.526916   
                            \checkmark $\times$    0.587745  0.534916   
                                       \checkmark  0.587745  0.534916   
                 \checkmark $\times$   $\times$    0.519761  0.529465   
                                       \checkmark  0.550995  0.527848   
                            \checkmark $\times$    0.580048  0.524546   
                                       \checkmark  0.582557  0.533863   

                                                      $F_1$  $P_{micro}$  \
Set    Model     RAG        LoRA       Reorder                             
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.015625     0.666667   
                                       \checkmark  0.005208     0.500000   
                            \checkmark $\times$    0.059721     0.961538   
                                       \checkmark  0.059721     0.961538   
                 \checkmark $\times$   $\times$    0.098323     0.846154   
                                       \checkmark  0.063403     0.692308   
                            \checkmark $\times$    0.059608     0.888889   
                                       \checkmark  0.055868     0.857143   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.511780     0.657143   
                                       \checkmark  0.508960     0.650943   
                            \checkmark $\times$    0.540149     0.700980   
                                       \checkmark  0.540149     0.700980   
                 \checkmark $\times$   $\times$    0.504969     0.621145   
                                       \checkmark  0.514691     0.616740   
                            \checkmark $\times$    0.531806     0.681373   
                                       \checkmark  0.537459     0.687500   

                                                   $R_{micro}$  $F_{1,micro}$  
Set    Model     RAG        LoRA       Reorder                                 
$\cap$ Hermes 3B $\times$   $\times$   $\times$       0.008696       0.017167  
                                       \checkmark     0.004348       0.008621  
                            \checkmark $\times$       0.108696       0.195312  
                                       \checkmark     0.108696       0.195312  
                 \checkmark $\times$   $\times$       0.143478       0.245353  
                                       \checkmark     0.117391       0.200743  
                            \checkmark $\times$       0.104348       0.186770  
                                       \checkmark     0.104348       0.186047  
$\cup$ Hermes 3B $\times$   $\times$   $\times$       0.600000       0.627273  
                                       \checkmark     0.600000       0.624434  
                            \checkmark $\times$       0.621739       0.658986  
                                       \checkmark     0.621739       0.658986  
                 \checkmark $\times$   $\times$       0.613043       0.617068  
                                       \checkmark     0.608696       0.612691  
                            \checkmark $\times$       0.604348       0.640553  
            

In [23]:
task_6_2_3_df[top_results:].style.highlight_max(axis=0, props="textbf:--rwrap;").format(precision=2).to_latex(
    report_dir / "task_6_2_3.tex",
    # float_format="%.2f",
    caption="Dev Set Result for Task 6.2.3 for various models and approaches.",
    label=f"tab:task:6_2_3{lbl_xtra}",
    clines="all;data",
    hrules=True
)
task_6_2_3_df

$P$       $R$  \
Set    Model     RAG        LoRA       Reorder                          
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.001302  0.003472   
                                       \checkmark  0.000000  0.000000   
                            \checkmark $\times$    0.056471  0.036594   
                                       \checkmark  0.056471  0.036594   
                 \checkmark $\times$   $\times$    0.107129  0.051278   
                                       \checkmark  0.019151  0.026367   
                            \checkmark $\times$    0.054417  0.027528   
                                       \checkmark  0.056013  0.029934   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.281717  0.287139   
                                       \checkmark  0.274650  0.287139   
                            \checkmark $\times$    0.330713  0.298445   
                                       \checkmark  0.330713  0.298445   
                 \checkmark $\times$   $\times$    0.280074  0.291721   
                                       \checkmark  0.302464  0.288745   
                            \checkmark $\times$    0.332182  0.288627   
                                       \checkmark  0.338642  0.295984   

                                                      $F_1$  $P_{micro}$  \
Set    Model     RAG        LoRA       Reorder                             
$\cap$ Hermes 3B $\times$   $\times$   $\times$    0.001894     0.023256   
                                       \checkmark  0.000000     0.000000   
                            \checkmark $\times$    0.040812     0.339713   
                                       \checkmark  0.040812     0.339713   
                 \checkmark $\times$   $\times$    0.059708     0.268382   
                                       \checkmark  0.020020     0.175758   
                            \checkmark $\times$    0.033838     0.306878   
                                       \checkmark  0.034570     0.319588   
$\cup$ Hermes 3B $\times$   $\times$   $\times$    0.254697     0.312789   
                                       \checkmark  0.248833     0.275815   
                            \checkmark $\times$    0.294700     0.364238   
                                       \checkmark  0.294700     0.364238   
                 \checkmark $\times$   $\times$    0.265903     0.277480   
                                       \checkmark  0.272644     0.271768   
                            \checkmark $\times$    0.293685     0.356282   
                                       \checkmark  0.299356     0.374564   

                                                   $R_{micro}$  $F_{1,micro}$  
Set    Model     RAG        LoRA       Reorder                                 
$\cap$ Hermes 3B $\times$   $\times$   $\times$       0.001786       0.003317  
                                       \checkmark     0.000000       0.000000  
                            \checkmark $\times$       0.126786       0.184655  
                                       \checkmark     0.126786       0.184655  
                 \checkmark $\times$   $\times$       0.130357       0.175481  
                                       \checkmark     0.103571       0.130337  
                            \checkmark $\times$       0.103571       0.154873  
                                       \checkmark     0.110714       0.164456  
$\cup$ Hermes 3B $\times$   $\times$   $\times$       0.362500       0.335815  
                                       \checkmark     0.362500       0.313272  
                            \checkmark $\times$       0.392857       0.378007  
                                       \checkmark     0.392857       0.378007  
                 \checkmark $\times$   $\times$       0.369643       0.316998  
                                       \checkmark     0.367857       0.312595  
                            \checkmark $\times$       0.369643       0.362840  
            

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tikzplotlib
import re

# style to sbn
plt.style.use("seaborn-v0_8")


def tikzplotlib_fix_ncols(obj):
    """
    workaround for matplotlib 3.6 renamed legend's _ncol to _ncols, which breaks tikzplotlib
    """
    if hasattr(obj, "_ncols"):
        obj._ncol = obj._ncols
    for child in obj.get_children():
        tikzplotlib_fix_ncols(child)


k = list(range(1, 20))


task_6_2_1_df_f1k = scoring_to_df(
    eval_submission_6_2_binary_tag_RE,
    "binary_tag_based_relations",
    k_limits_set=k,
    cap_gt=True,
    res_dir=results_dir / "hermes-3b-rag.json",
)
k_s=task_6_2_1_df_f1k.reset_index()['$k$'].to_numpy()
f1k=task_6_2_1_df_f1k["$F_{1,micro}$"]
p1k=task_6_2_1_df_f1k["$P_{micro}$"]
r1k=task_6_2_1_df_f1k["$R_{micro}$"]

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)
ax.plot(k_s, f1k, label="$F_{1,micro}$", color="blue")
ax.plot(k_s, p1k, label="$P_{micro}$", color="red")
ax.plot(k_s, r1k, label="$R_{micro}$", color="orange")
ax.set_ylabel("Score")
ax.set_xlabel("$k$")
ax.set_title("Scores @  $k$ for Task 6.2.1")
ax.legend()
ax.set_xticklabels(k_s)
tikzplotlib_fix_ncols(fig)
# tikzplotlib.save(
#     "f1_at_k_rerun.tex",
#     figure=fig,
#     axis_width="0.8\\linewidth",
#     axis_height="0.4\\linewidth",
#     # extra_axis_parameters={
#     #     "legend style": r"legend pos=outer north east",
#     #     "legend columns": 1,
#     #     "legend cell align": "left",
#     #     "legend to name": "loss_legend",
#     #     "legend entries": ["loss"],
#     # },
# )

Further mode: False


OSError: Error in opening the specified json file: ../data/merge_dev/hermes-3b-rag.json

In [ ]:
task_6_2_1_df_f1k.reset_index()['$k$']

NameError: name 'task_6_2_1_df_f1k' is not defined